## First, install the necessary libraries:

In [1]:
!pip install opencv-python face_recognition numpy pandas matplotlib seaborn scikit-learn

## Setup Environment

In [2]:
# Import necessary libraries
import cv2
import face_recognition
import numpy as np
import pandas as pd
import datetime
import os
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

# Set up directories for images and data
if not os.path.exists('employee_faces'):
    os.makedirs('employee_faces')

if not os.path.exists('attendance_logs'):
    os.makedirs('attendance_logs')


## Register Employees

In [ ]:
import cv2
import numpy as np
from IPython.display import display, clear_output
from PIL import Image
import os
import time

def register_employee():
    # Get employee details from input
    employee_id = input("Enter Employee ID: ")
    employee_name = input("Enter Employee Name: ")
    
    # Create the directory to save employee images if it doesn't exist
    if not os.path.exists('employee_faces'):
        os.makedirs('employee_faces')
    
    # Open the webcam
    video_capture = cv2.VideoCapture(0)  # 0 is usually the default webcam
    
    if not video_capture.isOpened():
        print("Error: Could not open webcam.")
        return
    
    print(f"Capturing face for {employee_name} (ID: {employee_id})... Image will be saved in 5 seconds.")
    
    start_time = time.time()  # Record the start time
    
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        
        if not ret:
            print("Failed to capture image.")
            break
        
        # Convert the frame to RGB (OpenCV uses BGR by default)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convert the frame to a PIL image for display in Jupyter Notebook
        img_pil = Image.fromarray(frame_rgb)
        
        # Display the frame in the notebook
        clear_output(wait=True)  # Clear previous frames
        display(img_pil)  # Display current frame
        
        # Check if 5 seconds have passed
        if time.time() - start_time > 5:
            # Save the current frame as the employee's image
            image_path = f"employee_faces/{employee_id}_{employee_name}.jpg"
            cv2.imwrite(image_path, frame)  # Save the image in the 'employee_faces' directory
            print(f"Image saved at {image_path}")
            break
    
    # Release the webcam
    video_capture.release()

# Call the function to start registration
register_employee()


## Facial Recognition for Check-In/Check-Out

In [4]:
import cv2
import face_recognition
import numpy as np
import os
import pandas as pd
from datetime import datetime, timedelta
import time

# Initialize video capture (0 for the default camera)
video_capture = cv2.VideoCapture(0)

# Load known faces and their names
known_faces = []
known_names = []
attendance_log = {}  # Dictionary to track check-in and check-out status

# Load your known faces and names from your dataset
def load_known_faces(known_faces_directory):
    for filename in os.listdir(known_faces_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image = face_recognition.load_image_file(os.path.join(known_faces_directory, filename))
            face_encoding = face_recognition.face_encodings(image)[0]
            known_faces.append(face_encoding)
            known_names.append(os.path.splitext(filename)[0])  # Use the filename as the name

load_known_faces('employee_faces')  # Update the path

# Function to log attendance to CSV (in the same row)
def log_attendance(employee_id, name, status):
    try:
        # Read the existing attendance log
        df = pd.read_csv("attendance.csv")
    except FileNotFoundError:
        # If file doesn't exist, create a new DataFrame with the required columns
        df = pd.DataFrame(columns=["ID", "Employee Name", "Date", "Time", "Status"])

    # Log the attendance entry
    now = datetime.now()
    date_str = now.strftime('%Y-%m-%d')
    time_str = now.strftime('%H:%M:%S')
    new_row = {
        "ID": employee_id,
        "Employee Name": name,
        "Date": date_str,
        "Time": time_str,
        "Status": status
    }
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)  # Use pd.concat instead of append
    df.to_csv("attendance.csv", index=False)  # Save the updated DataFrame back to the CSV

# Function to handle check-in and check-out process
def check_in_check_out():
    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        # Convert the image from BGR to RGB and resize for faster processing
        rgb_small_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_small_frame = cv2.resize(rgb_small_frame, (0, 0), fx=0.25, fy=0.25)

        # Find all face locations and encodings
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        for face_encoding, face_location in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_faces, face_encoding)
            name = "Unknown"
            if True in matches:
                first_match_index = matches.index(True)
                name = known_names[first_match_index]
                employee_id = name.split('_')[0]  # Extracting employee ID from name format

                print(f"Recognized: {name} (ID: {employee_id})")
                now = datetime.now()

                # Check if employee has already checked in
                if employee_id not in attendance_log:
                    # First time seeing the face today - log check-in
                    print(f"Checking in {name} (ID: {employee_id}). Please wait...")
                    time.sleep(np.random.randint(3, 9))  # Wait for 3 to 8 seconds for recognition
                    log_attendance(employee_id, name, status="Check-in")
                    attendance_log[employee_id] = {"check_in": now, "check_out": None}
                    print(f"{name} (ID: {employee_id}) checked in at {now.strftime('%H:%M:%S')}")

                elif employee_id in attendance_log:
                    check_in_time = attendance_log[employee_id]["check_in"]
                    check_out_time = attendance_log[employee_id]["check_out"]

                    # If already checked in, check-out process
                    if check_out_time is None:
                        if now - check_in_time >= timedelta(minutes=5):  # Must wait 5 min after check-in
                            print(f"Checking out {name} (ID: {employee_id}). Please wait...")
                            time.sleep(np.random.randint(3, 9))  # Wait for 3 to 8 seconds for recognition
                            log_attendance(employee_id, name, status="Check-out")
                            attendance_log[employee_id]["check_out"] = now
                            print(f"{name} (ID: {employee_id}) checked out at {now.strftime('%H:%M:%S')}")

                        else:
                            print(f"{name} (ID: {employee_id}) must wait before checking out.")
                    else:
                        # Already checked out - allow check-in after 5 min cooldown
                        if now - check_out_time >= timedelta(minutes=5):
                            print(f"Checking in {name} (ID: {employee_id}). Please wait...")
                            time.sleep(np.random.randint(3, 9))  # Wait for 3 to 8 seconds for recognition
                            log_attendance(employee_id, name, status="Check-in")
                            attendance_log[employee_id]["check_in"] = now
                            attendance_log[employee_id]["check_out"] = None  # Reset check-out status
                            print(f"{name} (ID: {employee_id}) checked in at {now.strftime('%H:%M:%S')}")
                        else:
                            print(f"{name} (ID: {employee_id}) must wait before checking in again.")

            # Draw a rectangle around the face
            top, right, bottom, left = [v * 4 for v in face_location]
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        # Display the video feed with the recognized names
        cv2.imshow('Video', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video capture and close windows
    video_capture.release()
    cv2.destroyAllWindows()

# Start the check-in/check-out system
check_in_check_out()


Recognized: 101_Rahul (ID: 101)
Checking in 101_Rahul (ID: 101). Please wait...
101_Rahul (ID: 101) checked in at 18:01:44
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 101) must wait before checking out.
Recognized: 101_Rahul (ID: 101)
101_Rahul (ID: 

KeyboardInterrupt: 

In [50]:
pip install --upgrade face_recognition opencv-python


## Spoof Detection

In [5]:
"""import cv2
import numpy as np
import face_recognition
import datetime
import os

# Directory to store attendance data
attendance_file = "attendance.csv"

# Function to check for spoofing (simple example)
def is_spoof(frame):
    # Implement your spoof detection logic here
    # This is a placeholder that always returns False (no spoof detected)
    return False

# Function to log attendance
def log_attendance(name, status):
    current_time = datetime.datetime.now()
    with open(attendance_file, 'a') as f:
        f.write(f"{name},{current_time},{status}\n")

# Main function for check-in and check-out
def check_in_check_out():
    # Load known face encodings and their names
    known_face_encodings = []
    known_face_names = []

    # Load your employee images here (Assuming images are named as ID_Name.jpg)
    employee_folder = "employee_faces"
    for filename in os.listdir(employee_folder):
        if filename.endswith(".jpg"):
            image_path = os.path.join(employee_folder, filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)
            known_face_names.append(os.path.splitext(filename)[0])  # Use filename without extension as name

    # Initialize video capture
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        
        # Convert the image from BGR to RGB
        rgb_small_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face locations
        face_locations = face_recognition.face_locations(rgb_small_frame, model="hog")
        
        if not face_locations:
            print("No faces detected.")
            continue

        # Detect face encodings based on the detected face locations
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        if not face_encodings:
            print("No face encodings found.")
            continue
        
        # Loop through each detected face
        for face_encoding, face_location in zip(face_encodings, face_locations):
            # Check if the detected face matches known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Check for spoofing
            if is_spoof(frame):
                print(f"Spoof detected for {name}!")
                continue
            
            # Log attendance (check-in)
            log_attendance(name, "Check-in")
            print(f"{name} checked in at {datetime.datetime.now()}.")

        # Display the resulting frame with the bounding box
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Show the frame
        cv2.imshow('Video', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video capture and close windows
    video_capture.release()
    cv2.destroyAllWindows()

# Run the check-in/check-out system
if __name__ == "__main__":
    # Ensure attendance file exists
    if not os.path.isfile(attendance_file):
        with open(attendance_file, 'w') as f:
            f.write("Name,Timestamp,Status\n")  # Write headers

    check_in_check_out()"""
 

'import cv2\nimport numpy as np\nimport face_recognition\nimport datetime\nimport os\n\n# Directory to store attendance data\nattendance_file = "attendance.csv"\n\n# Function to check for spoofing (simple example)\ndef is_spoof(frame):\n    # Implement your spoof detection logic here\n    # This is a placeholder that always returns False (no spoof detected)\n    return False\n\n# Function to log attendance\ndef log_attendance(name, status):\n    current_time = datetime.datetime.now()\n    with open(attendance_file, \'a\') as f:\n        f.write(f"{name},{current_time},{status}\n")\n\n# Main function for check-in and check-out\ndef check_in_check_out():\n    # Load known face encodings and their names\n    known_face_encodings = []\n    known_face_names = []\n\n    # Load your employee images here (Assuming images are named as ID_Name.jpg)\n    employee_folder = "employee_faces"\n    for filename in os.listdir(employee_folder):\n        if filename.endswith(".jpg"):\n            image_p

## Attendance log

In [6]:
import datetime
import pandas as pd
import os

# Create attendance_logs directory if it doesn't exist
if not os.path.exists('attendance_logs'):
    os.makedirs('attendance_logs')

def log_attendance(employee_id, name):
    current_time = datetime.datetime.now()
    date_str = current_time.strftime('%Y-%m-%d')
    time_str = current_time.strftime('%H:%M:%S')
    log_file = f'attendance_logs/{date_str}.csv'

    # Determine if the employee can check in or check out
    if check_already_checked_out(employee_id, log_file):
        status = 'Check-in'
    else:
        status = 'Check-out'

    # Log entry with ID, Name, Date, Time, and Status (Check-in/Check-out)
    log_entry = {
        'ID': employee_id,
        'Name': name,
        'Date': date_str,
        'Time': time_str,
        'Status': status
    }

    # Append entry to the log CSV
    try:
        log_df = pd.DataFrame([log_entry])
        log_df.to_csv(log_file, mode='a', header=not os.path.exists(log_file), index=False)
        print(f"Attendance logged for {name} (ID: {employee_id}) at {time_str} ({status})")
    except Exception as e:
        print(f"Error logging attendance: {e}")

def check_already_checked_out(employee_id, log_file):
    """Check if the user has already checked out today."""
    try:
        if os.path.exists(log_file):
            # Read today's attendance log
            log_df = pd.read_csv(log_file)
            # Check the latest entry for the employee ID
            if not log_df[log_df['ID'] == employee_id].empty:
                last_entry = log_df[log_df['ID'] == employee_id].iloc[-1]
                # If the last entry is 'Check-in', return True
                return last_entry['Status'] == 'Check-out'
    except Exception as e:
        print(f"Error checking attendance: {e}")
    return True  # Assume true if file doesn't exist or no entry found

 

## Shift & Overtime Tracking

In [8]:
import pandas as pd

def calculate_overtime(attendance_df):
    # Ensure the 'Time' column is in datetime format
    attendance_df['Time'] = pd.to_datetime(attendance_df['Time'])

    # Sort the dataframe by Name and Time to ensure proper calculation
    attendance_df = attendance_df.sort_values(by=['Name', 'Time'])

    # Extract date component for analysis
    attendance_df['Date'] = attendance_df['Time'].dt.date

    # Group by employee and date to calculate daily work hours
    work_hours_per_day = []
    
    grouped = attendance_df.groupby(['Name', 'Date'])
    
    for (name, date), group in grouped:
        # Sort by time to get in-out pairs
        group = group.sort_values('Time')
        
        # Filter out "Check-in" and "Check-out" pairs
        check_in_times = group[group['Status'] == 'Check-in']['Time']
        check_out_times = group[group['Status'] == 'Check-out']['Time']
        
        # Ensure equal number of check-ins and check-outs
        total_worked_seconds = 0
        for check_in, check_out in zip(check_in_times, check_out_times):
            total_worked_seconds += (check_out - check_in).total_seconds()
        
        # Convert seconds to hours
        total_worked_hours = total_worked_seconds / 3600
        
        # Log daily work hours
        work_hours_per_day.append({
            'Name': name,
            'Date': date,
            'WorkedHours': total_worked_hours
        })
    
    # Create a DataFrame from the daily work hours
    work_hours_df = pd.DataFrame(work_hours_per_day)
    
    # Determine overtime (greater than 8 hours)
    work_hours_df['Overtime'] = work_hours_df['WorkedHours'] - 8
    overtime_df = work_hours_df[work_hours_df['Overtime'] > 0]
    
    return overtime_df

# Example usage:
# Assuming attendance_data is your attendance DataFrame with the necessary columns.
attendance_data = pd.DataFrame({
    'ID': ['1001', '1001', '1001', '1001', '1001', '1001'],
    'Name': ['Rahul Kumar', 'Rahul Kumar', 'Rahul Kumar', 'Rahul Kumar', 'Rahul Kumar', 'Rahul Kumar'],
    'Date': ['2024-10-05'] * 6,
    'Time': ['2024-10-05 09:00:00', '2024-10-05 17:00:00', 
             '2024-10-05 09:05:00', '2024-10-05 17:05:00', 
             '2024-10-05 09:10:00', '2024-10-05 17:10:00'],
    'Status': ['Check-in', 'Check-out', 
               'Check-in', 'Check-out', 
               'Check-in', 'Check-out']
})

# Convert time strings to datetime
attendance_data['Time'] = pd.to_datetime(attendance_data['Time'])

# Calculate overtime
overtime_df = calculate_overtime(attendance_data)

# Display the results
print(overtime_df)


          Name        Date  WorkedHours  Overtime
0  Rahul Kumar  2024-10-05         24.0      16.0


## Generate Attendance Reports

In [30]:
import pandas as pd

def generate_attendance_report(csv_file):
    """
    Generates an attendance report based on check-in/check-out times from your CSV format.
    
    Parameters:
    - csv_file (str): Path to the CSV file containing attendance logs.
    
    Returns:
    - A pandas DataFrame containing the report.
    """
    # Read the attendance data from the CSV file
    try:
        attendance_data = pd.read_csv(csv_file, header=None, names=["ID", "Employee Name", "Date", "Time", "Status"])
    except Exception as e:
        print(f"Error reading the CSV file: {e}")
        return None

    # Check if the data was read correctly
    if attendance_data.empty or attendance_data.shape[0] <= 1:  # Ensuring there's data beyond the header
        print("The CSV file is empty or the format is incorrect.")
        return None

    # Debug: Display the first few rows of the data
    print("Data preview:")
    print(attendance_data.head())

    # Convert 'Date' and 'Time' to datetime for easier manipulation
    try:
        # Specify the date and time formats explicitly
        attendance_data['Timestamp'] = pd.to_datetime(
            attendance_data['Date'] + ' ' + attendance_data['Time'],
            format='%Y-%m-%d %H:%M:%S',
            errors='coerce'  # This will convert unparseable formats to NaT
        )
        # Drop rows where the timestamp conversion failed
        attendance_data.dropna(subset=['Timestamp'], inplace=True)
    except Exception as e:
        print(f"Error converting date and time to datetime: {e}")
        return None

    # Separate the check-ins and check-outs
    check_in_data = attendance_data[attendance_data['Status'].str.lower() == 'check-in']
    check_out_data = attendance_data[attendance_data['Status'].str.lower() == 'check-out']

    # Debug: Display check-in and check-out data
    print("Check-in Data Preview:")
    print(check_in_data)
    
    print("Check-out Data Preview:")
    print(check_out_data)

    # Merge the check-in and check-out data based on 'Employee Name' and the nearest timestamps
    report = pd.merge_asof(check_in_data.sort_values('Timestamp'), 
                           check_out_data.sort_values('Timestamp'), 
                           on='Timestamp', 
                           by='Employee Name', 
                           direction='forward', 
                           suffixes=('_in', '_out'))

    # Debug: Display the merged report
    print("Merged Report Preview:")
    print(report)

    # Debug: Print the actual column names in the merged report
    print("Merged report columns:", report.columns.tolist())

    # Check if merged report has 'Timestamp_out'
    if 'Timestamp_out' not in report.columns:
        print("Warning: No matching check-out records found for some check-ins.")
        # Return only available columns
        return report[['ID_in', 'Employee Name', 'Date_in', 'Time_in']]

    # Calculate the total working hours by subtracting the check-in time from the check-out time
    report['Total_Working_Hours'] = (report['Timestamp_out'] - report['Timestamp_in']).dt.total_seconds() / 3600  # Convert to hours

    # Return the attendance report with relevant columns
    return report[['ID_in', 'Employee Name', 'Date_in', 'Timestamp_in', 'Timestamp_out', 'Total_Working_Hours']]

# Example usage: generate the attendance report
attendance_report = generate_attendance_report('attendance.csv')

# Display the report
if attendance_report is not None:
    print(attendance_report)


Data preview:
    ID  Employee Name        Date      Time     Status
0   ID  Employee Name        Date      Time     Status
1  101      101_Rahul  2024-10-06  03:23:11   Check-in
2  101      101_Rahul  2024-10-06  03:28:36  Check-out
3  101      101_Rahul  2024-10-06  03:35:15   Check-in
Check-in Data Preview:
    ID Employee Name        Date      Time    Status           Timestamp
1  101     101_Rahul  2024-10-06  03:23:11  Check-in 2024-10-06 03:23:11
3  101     101_Rahul  2024-10-06  03:35:15  Check-in 2024-10-06 03:35:15
Check-out Data Preview:
    ID Employee Name        Date      Time     Status           Timestamp
2  101     101_Rahul  2024-10-06  03:28:36  Check-out 2024-10-06 03:28:36
Merged Report Preview:
  ID_in Employee Name     Date_in   Time_in Status_in           Timestamp  \
0   101     101_Rahul  2024-10-06  03:23:11  Check-in 2024-10-06 03:23:11   
1   101     101_Rahul  2024-10-06  03:35:15  Check-in 2024-10-06 03:35:15   

  ID_out    Date_out  Time_out Status_out 

In [38]:
import pandas as pd

def load_attendance_data(csv_file):
    # Load attendance data from CSV file
    return pd.read_csv(csv_file)

def generate_attendance_report(csv_file):
    attendance_data = load_attendance_data(csv_file)

    # Check if 'Status' column exists
    if 'Status' not in attendance_data.columns:
        raise KeyError("The column 'Status' is missing from the attendance data.")

    # Separate check-in and check-out records
    check_in_data = attendance_data[attendance_data['Status'] == 'Check-in']
    check_out_data = attendance_data[attendance_data['Status'] == 'Check-out']

    # Merge check-in and check-out data
    report = pd.merge(check_in_data, check_out_data, on='Name', suffixes=('_in', '_out'))
    
    # Convert timestamps to datetime
    report['Timestamp_in'] = pd.to_datetime(report['Timestamp_in'], errors='coerce')
    report['Timestamp_out'] = pd.to_datetime(report['Timestamp_out'], errors='coerce')

    # Calculate total working hours
    report['Total_Working_Hours'] = (report['Timestamp_out'] - report['Timestamp_in']).dt.total_seconds() / 3600  # Convert to hours

    return report[['Name', 'Timestamp_in', 'Timestamp_out', 'Total_Working_Hours']]

def generate_daily_report(attendance_data, date):
    daily_data = attendance_data[attendance_data['Timestamp_in'].dt.date == pd.to_datetime(date).date()]
    shift_start_time = pd.to_datetime('09:00:00').time()  # Adjust as necessary

    late_arrivals = daily_data[daily_data['Timestamp_in'].dt.time > shift_start_time]
    early_departures = daily_data[daily_data['Timestamp_out'].dt.time < pd.to_datetime('17:00:00').time()]  # Adjust as necessary

    report = {
        'Total_Attendance': len(daily_data),
        'Late_Arrivals': len(late_arrivals),
        'Early_Departures': len(early_departures),
        'Total_Working_Hours': daily_data['Total_Working_Hours'].sum()
    }
    return report

def generate_weekly_report(attendance_data, date):
    date = pd.to_datetime(date)
    week_start = date - pd.Timedelta(days=date.weekday())  # Start of the week (Monday)
    week_end = week_start + pd.Timedelta(days=6)  # End of the week (Sunday)

    weekly_data = attendance_data[(attendance_data['Timestamp_in'] >= week_start) & (attendance_data['Timestamp_in'] <= week_end)]

    report = {
        'Total_Attendance': len(weekly_data),
        'Total_Working_Hours': weekly_data['Total_Working_Hours'].sum()
    }
    return report

def generate_monthly_report(attendance_data, date):
    month_start = pd.to_datetime(date).replace(day=1)
    next_month_start = (month_start + pd.DateOffset(months=1)).replace(day=1)

    monthly_data = attendance_data[(attendance_data['Timestamp_in'] >= month_start) & (attendance_data['Timestamp_in'] < next_month_start)]

    report = {
        'Total_Attendance': len(monthly_data),
        'Total_Working_Hours': monthly_data['Total_Working_Hours'].sum()
    }
    return report

# Example usage
csv_file = 'attendance.csv'  # Update the CSV file path as needed
attendance_report = generate_attendance_report(csv_file)

# Generate reports for a specific date
date = '2024-10-06'
daily_report = generate_daily_report(attendance_report, date)
print("Daily Report:", daily_report)

weekly_report = generate_weekly_report(attendance_report, date)
print("Weekly Report:", weekly_report)

monthly_report = generate_monthly_report(attendance_report, date)
print("Monthly Report:", monthly_report)


KeyError: 'Name'

## Data Analysis & Insights

In [40]:
def analyze_trends(attendance_df):
    # Calculate absenteeism trends
    absent_days = attendance_df.groupby('Name')['Date'].nunique()

    # Plot absenteeism
    absent_days.plot(kind='bar')
    plt.title('Absenteeism Trends')
    plt.show()

# Load sample attendance data
attendance_df = pd.read_csv('attendance_logs/2024-10-02.csv')
analyze_trends(attendance_df)


FileNotFoundError: [Errno 2] No such file or directory: 'attendance_logs/2024-10-02.csv'